In [20]:
import math 
import pandas as pd
import random 
import numpy as np
import pickle

In [21]:
# Setting simulation parameters values

N = [30]  # population size
D = [1] # Densities
T = [2, 7, 10] # infectious time in days

d = 2 # distance in meters 
iter = 1 # number of iterations
time = 100 # exposure time in minutes
T_max = 10


In [ ]:
# importing trace

with open(r"D:\Manal\Programming\project1\R0_simulation\trace_enveloppe_2m_0.1.pkl", 'rb') as f:
    trace_env = pickle.load(f)

# extract samples from the trace

samples = trace_env.posterior.stack(sample=("chain", "draw"))

# select a random sample from the trace
selected_samples = np.random.choice(range(len(samples['p0'])), size=100)

# extract the values of the parameters
a_sample = samples['p0'].values[selected_samples]
b_sample = samples['lambda'].values[selected_samples]


In [ ]:
%%time 

# Perform the simulations

data = []
results = dict()# using dictionary instead of  lists
for k in D: # density
    for n in N: # population size 
            for i in range(iter): # iterations
                p0 = p0_sample[i]
                lambda_ = lambda_sample[i]
                temp_results = []

                for m in range(n): # individuals
                    mu = np.random.uniform(0, 1) # a random number between 0 and 1
                    Rmax= np.sqrt(n/(k * np.pi)) # maximum radius distance
                    r0 = np.sqrt((n * mu)/(k * np.pi)) # generate initial radius distance r0
                    r = r0
                    p = 1 - (p0 * np.exp(-lambda_ * r0)) 
                    proba_minute = p
                    
                    for j in range(time):
                        if j == 0:
                            pass
                        else:
                            theta_ =  np.random.uniform(0, 2*np.pi) # a random number between 0 and pi
                            r = min(np.sqrt(r**2 + d**2 - 2 * r * d * np.cos(theta_) ),Rmax) # generate next radius distance per minute 
                            p = 1 - (p0 * np.exp(-lambda_ * r)) # probability per minute
                            proba_minute *= p # product on all probabilities per minute for each ID
                    
                    temp_results.append(proba_minute)
                    

                #status = [np.random.binomial(T_max, 1 - val) for val in temp_results]
   
                for t in T:
                        status = [np.random.binomial(t, 1 - val) for val in temp_results] # Bernoulli drawing for infectious status (0 or 1) using the final proba
                        a =[k, n, t] # creating the key for the dictionary
                        key = str(a) # writing as a string
                        try:
                            results[key].append(n - status.count(0))
                        except KeyError:
                            results[key]=[n - status.count(0)]
                        

In [ ]:
results

In [ ]:
with open(r"D:\Manal\Programming\project1\R0_simulation\results_dictionary.pkl", 'wb') as f:
    pickle.dump(results, f)

In [ ]:
triplets = list(results.keys())

In [ ]:
col_names = ['Density', 'Population_size', 'Infectious_time'] + [f'iter_{i}' for i in range(1, max(len(value) for value in results.values()) + 1)]



In [ ]:
df = pd.DataFrame(
    [[float(triplet[1:-1].split(', ')[0])] + [int(x) for x in triplet[1:-1].split(', ')[1:]] + value for triplet, value in results.items()],
    columns=col_names
)

In [ ]:
df

In [ ]:
# Save the simulation results into a csv file

df.to_csv("simulation_results_v5.csv", index=False)